In [1]:
from stonks.data.data_reader import MktDataReader
from stonks.backtest.backtest import BackTester
from stonks.backtest.strategy.constant_equal_weights import EqualWeightsStrategy

import functools
import math
import backtrader as bt
import backtrader.feeds as btfeeds

In [2]:
tickers = [
    'AAPL', # Apple -> tech
    'TSLA', # Tesla -> tech
    'GOOGL', # Google -> tech
    'AMZN', # Amazon -> consumer/tech
    'MA', # MasterCard -> finance
    'V', # Visa -> finance
    'MELI', # MercadoLibre -> consumer
    'NVDA', # Nvidia -> chips
    'ASML', # ASML -> chips
    'FB' # facebook -> social media
]

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2008-01-01'
end_date = '2021-04-05'

dr = MktDataReader(start_date, end_date, 'yahoo', tickers)

[*********************100%***********************]  10 of 10 completed


In [3]:
# impute missing data
dr.impute_missing_data()
# fetch market caps
dr.fetch_market_caps()

In [4]:
dr.df_stocks_bkfilled.head(4)

,AAPL,AMZN,ASML,FB,GOOGL,MA,MELI,NVDA,TSLA,V
Date,,,,,,,,,,
2007-12-31,6.074097,92.639999,21.576687,38.23,346.086090,20.153185,71.589546,7.815117,4.778,12.871653
2008-01-02,5.974743,96.250000,21.011238,38.23,342.937927,20.104492,67.926750,7.583099,4.778,12.871653
2008-01-03,5.977503,95.209999,20.652660,38.23,343.007996,19.864748,67.287216,7.523369,4.778,12.871653
2008-01-04,5.521210,88.790001,19.466599,38.23,328.828827,18.841169,62.103062,6.891638,4.778,12.871653


In [5]:
#dr.df_stocks_bkfilled.iloc[:10,:]
len(dr.df_stocks_bkfilled.columns)

10

In [6]:
bt = BackTester(dr.df_stocks_bkfilled, 0, 5)
bt.set_strategy(EqualWeightsStrategy())

# manual iteration through the generator
# bt.next_per()
# bt.next_per()
# bt.next_per()
# bt.next_per()

# iterate all
bt.backtest()

668


In [15]:
len(bt.data.columns)

10

In [16]:
bt.enr_df.iloc[:,1:11]

,AAPL,AMZN,ASML,FB,GOOGL,MA,MELI,NVDA,TSLA,V
Date,,,,,,,,,,
2007-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-02,-0.016357,0.038968,-0.026206,0.0,-0.009096,-0.002416,-0.051164,-0.029688,0.0,0.0
2008-01-03,0.000462,-0.010805,-0.017066,0.0,0.000204,-0.011925,-0.009415,-0.007877,0.0,0.0
2008-01-04,-0.076335,-0.067430,-0.057429,0.0,-0.041338,-0.051527,-0.077045,-0.083969,0.0,0.0
2008-01-07,-0.013385,0.000338,-0.002125,0.0,-0.011796,-0.003380,-0.042440,-0.103333,0.0,0.0


In [8]:
ff = bt.enr_df.copy()
ff

,weights,AAPL,AMZN,ASML,FB,GOOGL,MA,MELI,NVDA,TSLA,V,portfolio_daily_returns,cumulative_returns
Date,,,,,,,,,,,,,
2007-12-31,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-02,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.016357,0.038968,-0.026206,0.0,-0.009096,-0.002416,-0.051164,-0.029688,0.0,0.0,-0.009596,0.990404
2008-01-03,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",0.000462,-0.010805,-0.017066,0.0,0.000204,-0.011925,-0.009415,-0.007877,0.0,0.0,-0.005642,0.984816
2008-01-04,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.076335,-0.067430,-0.057429,0.0,-0.041338,-0.051527,-0.077045,-0.083969,0.0,0.0,-0.045507,0.940000
2008-01-07,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.013385,0.000338,-0.002125,0.0,-0.011796,-0.003380,-0.042440,-0.103333,0.0,0.0,-0.017612,0.923444


In [877]:
from abc import ABC, abstractmethod

class BaseStrategy(ABC):
    def __init__(self):
        # ideally this should be an abstract property
        self.strat_executed = False
        self.weights = None
    
    @abstractmethod
    def execute(self):
        pass
    
    def execute_strategy(self):
        self.execute()
    
    def set_period_data(self, curr_period_data):
        self.curr_period_data = curr_period_data
    
class EqualWeightsStrategy(BaseStrategy):    
    # override of the abstract method
    def execute(self):
        if self.curr_period_data.empty == False:
            self.log()
            self.set_weights()
            self.compute_returns()
            
#             print(f"pnl is: {self.rets}")
#             print(f"weights are: {self.weights_array.reshape(-1,1)}")
#             self.daily_pnl = np.dot(self.rets, self.weights_array.reshape(-1,1))  # works mathematically accurate way
            
            
            self.strat_executed = True

    def set_weights(self):
        dts = self.curr_period_data.index
        n = len(self.curr_period_data.columns)
        weights_lst = [1/n for _ in range(n)]
        weights_lst_rep = [weights_lst for _ in range(len(dts))]

        self.weights = pd.DataFrame(
            {
                'weights': weights_lst_rep
            },
            index = dts
        )
#         self.num_tickers = len(self.curr_period_data.columns)
#         self.weights_lst = [1/self.num_tickers for _ in range(self.num_tickers)]
#         self.weights_array = np.array(self.weights_lst)
        
        # create a matrix of equally sized weights
#         n = self.curr_period_data.shape[0]
#         lst = [weights_array for _ in range(n)]
        
        # weights matrix
#         self.weights = functools.reduce(lambda a, b: np.vstack((a, b)), lst)
#         weights_matrix = np.vstack((weights_array.reshape(1,-1), weights_array.reshape(1,-1)))
        
    def compute_returns(self):
#         self.rets = ((self.curr_period_data - self.curr_period_data.shift(1)) / self.curr_period_data)[1:]
        self.rets = self.curr_period_data.pct_change().iloc[1:,:]

    def log(self):
        print(f"The shape of the current data is: {self.curr_period_data.shape}")
#         print(f"{self.elapsed_period_data.empty}")
        print(f"Dt from: {self.curr_period_data.index[0].strftime('%Y-%m-%d')} to: {self.curr_period_data.index[-1].strftime('%Y-%m-%d')}") if self.curr_period_data.empty != True else print("The DF is empty")
        

In [931]:
class BackTester():
    def __init__(self, data, initial_period, window_size):
        self.data = data
        self._rts = data.pct_change()
        self._data_generator = self.df_generator()  # data generator that will be used to traverse the df with the next method
        self._generator_iteration = 0
        
        self.initial_period = initial_period
        self.window_size = window_size
        
        self.log = None
        
        # portfolio based properties
        self.initial_capital = 100
        self.weights_df = self._rts.copy()
#         self.weights = []

    def backtest(self):
        print("Calculating Strategy for the period")
    
    def df_generator(self):
        window_selection = 0 + self.initial_period  # initial selection of the data
        while(True):
            yield self.data.iloc[:window_selection,:]
            window_selection += self.window_size  # advance the generator one window size
            self._generator_iteration += 1
            
    def set_strategy(self, strategy):
        self.strategy = strategy  # you'd need to overwrite strategy before instantiating the class
    
    def compute_metrics(self, weights):
#         self.enr_df = weights.merge(self.weights_df, left_index=True, right_index=True, how = "right") # join with the data
        self.enr_df = weights.merge(self.weights_df, left_index=True, right_index=True) # join with the data
    
        # this gets called everytime you compute the metrics should be only once at the end
        self.enr_df["portfolio_daily_returns"] = self.enr_df.apply(lambda row: np.dot(np.array(row[:1][0]), row[1:len(ff.columns)].values), axis = 1)
        self.enr_df["cumulative_returns"] = (self.enr_df["portfolio_daily_returns"] + 1).cumprod()

        
        #print(daily_rts_weights_adj[0,:] * self.initial_capital)
        
#         cum_rets_per = (daily_pnl_df + 1).cumprod()
#         total_returns_period = (daily_pnl_df + 1).prod() * self.initial_capital
#         #print(cum_rets_per * self.initial_capital)
#         print(f"Portfolio value at the end of the period: {total_returns_period: .2f}")

    def next_per(self):
        assert self.strategy != None, "Set a strategy first"
        print("Executing the strategy...")
        next_df = next(self._data_generator)
        print(f"Setting the data with shape: {next_df.shape}, generator iteration {self._generator_iteration}")
        self.strategy.set_period_data(next_df)
        self.strategy.execute()
        
        if self.strategy.strat_executed:
            weights = self.strategy.weights
#             prices = self.strategy.curr_period_data.values
            self.compute_metrics(weights)
    
    def run(self):
        n_windows = math.ceil(self.data.shape[0] / self.window_size)
        print(n_windows)
        
        for _ in range(n_windows)[:2]:
            self.next_per()
    
# the abstract class needs to be the strategy or the logger
# from abc import ABC, abstractmethod

# class Strategy(ABC):
#     @abstractmethod
#     def log(self):
#         pass

In [932]:
bt = BackTester(dr.df_stocks_bkfilled, 0, 5)
bt.set_strategy(EqualWeightsStrategy())

# manual iteration through the generator
# bt.next_per()
# bt.next_per()
# bt.next_per()
# bt.next_per()

# iterate all
bt.run()

668
Executing the strategy...
Setting the data with shape: (0, 10), generator iteration 0
Executing the strategy...
Setting the data with shape: (5, 10), generator iteration 1
The shape of the current data is: (5, 10)
Dt from: 2007-12-31 to: 2008-01-07


In [933]:
ff = bt.enr_df.copy()
# ff["cumulative_returns"] = (ff["portfolio_daily_returns"] + 1).cumprod()
# aa_1 = (1+aa.iloc[1:,:]).cumprod()
# aa

In [934]:

ff

,weights,AAPL,AMZN,ASML,FB,GOOGL,MA,MELI,NVDA,TSLA,V,portfolio_daily_returns,cumulative_returns
Date,,,,,,,,,,,,,
2007-12-31,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-02,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.016357,0.038968,-0.026206,0.0,-0.009096,-0.002416,-0.051164,-0.029688,0.0,0.0,-0.009596,0.990404
2008-01-03,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",0.000462,-0.010805,-0.017066,0.0,0.000204,-0.011925,-0.009415,-0.007877,0.0,0.0,-0.005642,0.984816
2008-01-04,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.076335,-0.067430,-0.057429,0.0,-0.041338,-0.051527,-0.077045,-0.083969,0.0,0.0,-0.045507,0.940000
2008-01-07,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.013385,0.000338,-0.002125,0.0,-0.011796,-0.003380,-0.042440,-0.103333,0.0,0.0,-0.017612,0.923444


In [915]:
import math

dd = dr.df_stocks_bkfilled.copy()
window_size = 200
math.ceil(dd.shape[0] / 200)

17

In [851]:
ff = bt.enr_df.copy()
ff["portfolio_daily_returns"] = ff.apply(lambda row: np.dot(np.array(row[:1][0]), row[1:len(ff.columns)].values), axis = 1)
# drr = ff.iloc[:,1:].apply(lambda x: x, axis = 1)

In [852]:
ff

,weights,AAPL,AMZN,ASML,FB,GOOGL,MA,MELI,NVDA,TSLA,V,portfolio_daily_returns
Date,,,,,,,,,,,,
2007-12-31,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-02,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.016357,0.038968,-0.026206,0.0,-0.009096,-0.002416,-0.051164,-0.029688,0.0,0.0,-0.009596
2008-01-03,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",0.000462,-0.010805,-0.017066,0.0,0.000204,-0.011925,-0.009415,-0.007877,0.0,0.0,-0.005642
2008-01-04,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.076335,-0.067430,-0.057429,0.0,-0.041338,-0.051527,-0.077045,-0.083969,0.0,0.0,-0.045507
2008-01-07,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.013385,0.000338,-0.002125,0.0,-0.011796,-0.003380,-0.042440,-0.103333,0.0,0.0,-0.017612
2008-01-08,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.035972,-0.010583,-0.011004,0.0,-0.027062,-0.012518,-0.004399,0.021189,0.0,0.0,-0.008035
2008-01-09,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",0.047591,-0.030269,0.015434,0.0,0.034068,-0.006419,-0.006874,0.051693,0.0,0.0,0.010523
2008-01-10,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.007692,-0.011265,-0.018381,0.0,-0.009905,-0.003001,0.055702,-0.020768,0.0,0.0,-0.001531
2008-01-11,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",-0.029940,-0.037740,-0.032049,0.0,-0.013112,-0.085820,-0.022791,-0.043832,0.0,0.0,-0.026529


In [696]:
n = df_rets.shape[0]
import functools

lst = [weights_array for _ in range(n)]
# lst[0]

weights_matrix = functools.reduce(lambda a, b: np.vstack((a, b)), lst)

In [905]:


#df_rets['portfolio_daily_returns'] = df_rets.dot(weights_array)

In [887]:
g = (i for i in range(5))
next(g)
next(g)

1

In [900]:

# i = iter(range(10))
# while (x := next(i, None)) is not None and x < 5:
#     print(x)

In [537]:
# np.dot(weights, rets.values.T)

In [74]:
# Create a Stratey
class LogStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

#         self.dataclose = self.datas[0].close
#         self.dataclose_n = self.datas[1].close

    def next(self):
        # Simply log the closing price of the series from the reference
        
        # log the first price
        self.log('Close, %.2f' % self.dataclose)
        
        # log the second price
        self.log('Close, %.2f' % self.dataclose_n)


In [82]:
dat = bt.feeds.PandasData(dataname = tmp, name = tk)